# Day la Giao dien (GUI) de xu ly file Phase Space from Step-by-Step

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tkinter import Tk, filedialog, Button, Label

def analyze_files():
    # Hiển thị hộp thoại chọn thư mục
    folder_path = filedialog.askdirectory()
    if not folder_path:
        return
    
    # Tạo danh sách để lưu trữ kết quả
    results = []
    file_names = []
    dsbs_values = []
    ssbs_values = []
    ssb_dsb_ratios = []

    # Duyệt qua tất cả các tệp trong thư mục
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.phsp'):  # Chỉ xử lý các tệp có đuôi .phsp
            file_path = os.path.join(folder_path, file_name)
            
            # Đọc tệp vào DataFrame
            columns = [
                "Energy_imparted_per_event", "Dose_per_event_Gy", "DSB/Gy/Gbp", "SSB/Gy/Gbp", 
                "SB/Gy/Gbp", "SSB+/Gy/Gbp", "DSB+/Gy/Gbp", "MoreComplexDamage/Gy/Gbp", 
                "BD/Gy/Gbp", "DSBs", "DSBs_Direct", "DSBs_Indirect", "DSBs_Hybrid", 
                "DSBs_Direct_WithOneQuasiDirect", "DSBs_Direct_WithBothQuasiDirect", 
                "DSBs_Hybrid_WithOneQuasiDirect", "SSBs", "SSBs_Direct", "SSBs_QuasiDirect", 
                "SSBs_Indirect", "SBs", "SBs_Direct", "SBs_QuasiDirect", "SBs_Indirect", 
                "SSB+s", "DSB+s", "More_complex_damages", "BDs", "BDs_Direct", "BDs_QuasiDirect", 
                "BDs_Indirect", "Foci_150nm", "Foci_500nm"
            ]
            df = pd.read_csv(file_path, delim_whitespace=True, names=columns)
            
            # Chọn các cột mà bạn muốn so sánh
            selected_columns = [
                "Dose_per_event_Gy", "DSBs", "DSBs_Direct", "DSBs_Indirect", "DSBs_Hybrid", 
                "SSBs", "SSBs_Direct", "SSBs_QuasiDirect", "SSBs_Indirect", "DSB/Gy/Gbp", "SSB/Gy/Gbp"
            ]
            df_selected = df[selected_columns]

            total_dose = df_selected["Dose_per_event_Gy"].sum()
            total_DSBs = df_selected["DSBs"].sum()
            total_SSBs = df_selected["SSBs"].sum()

            total_DSBs_Direct = df_selected["DSBs_Direct"].sum()
            total_DSBs_Indirect = df_selected["DSBs_Indirect"].sum()
            total_DSBs_Hybrid = df_selected["DSBs_Hybrid"].sum()

            total_SSBs_Direct = df_selected["SSBs_Direct"].sum()
            total_SSBs_QuasiDirect = df_selected["SSBs_QuasiDirect"].sum()
            total_SSBs_Indirect = df_selected["SSBs_Indirect"].sum()
            
            ssb_dsb_ratio = total_SSBs / total_DSBs if total_DSBs != 0 else 0

            # Tính phần trăm cho các thành phần của DSBs và SSBs
            percentage_DSBs_Direct = (total_DSBs_Direct / total_DSBs) * 100 if total_DSBs != 0 else 0
            percentage_DSBs_Indirect = (total_DSBs_Indirect / total_DSBs) * 100 if total_DSBs != 0 else 0
            percentage_DSBs_Hybrid = (total_DSBs_Hybrid / total_DSBs) * 100 if total_DSBs != 0 else 0
            
            percentage_SSBs_Direct = (total_SSBs_Direct / total_SSBs) * 100 if total_SSBs != 0 else 0
            percentage_SSBs_QuasiDirect = (total_SSBs_QuasiDirect / total_SSBs) * 100 if total_SSBs != 0 else 0
            percentage_SSBs_Indirect = (total_SSBs_Indirect / total_SSBs) * 100 if total_SSBs != 0 else 0

            file_name = os.path.basename(file_path)
            file_names.append(file_name)
            dsbs_values.append(total_DSBs)
            ssbs_values.append(total_SSBs)
            ssb_dsb_ratios.append(ssb_dsb_ratio)

            # Thêm kết quả vào bảng so sánh
            stats = {
                'File': file_name,
                'total_dose': total_dose,
                'Number_DSBs': total_DSBs,
                'Number_DSBs_Direct': total_DSBs_Direct,
                'Number_DSBs_Indirect': total_DSBs_Indirect,
                'Number_DSBs_Hybrid': total_DSBs_Hybrid,
                'Number_SSBs': total_SSBs,
                'Number_SSBs_Direct': total_SSBs_Direct,
                'Number_SSBs_QuasiDirect': total_SSBs_QuasiDirect,
                'Number_SSBs_Indirect': total_SSBs_Indirect,
                'Percentage_DSBs_Direct': percentage_DSBs_Direct,
                'Percentage_DSBs_Indirect': percentage_DSBs_Indirect,
                'Percentage_DSBs_Hybrid': percentage_DSBs_Hybrid,
                'Percentage_SSBs_Direct': percentage_SSBs_Direct,
                'Percentage_SSBs_QuasiDirect': percentage_SSBs_QuasiDirect,
                'Percentage_SSBs_Indirect': percentage_SSBs_Indirect,
                'DSB/Gy/Gbp': df_selected["DSB/Gy/Gbp"].sum(),
                'SSB/Gy/Gbp': df_selected["SSB/Gy/Gbp"].sum(),
                'SSB/DSBs Ratio': ssb_dsb_ratio
            }
            
            results.append(stats)

            # Vẽ biểu đồ hình tròn nếu các giá trị hợp lệ
            sizes_DSBs = [percentage_DSBs_Direct, percentage_DSBs_Indirect, percentage_DSBs_Hybrid]
            sizes_SSBs = [percentage_SSBs_Direct, percentage_SSBs_QuasiDirect, percentage_SSBs_Indirect]
            
            if not any(np.isnan(sizes_DSBs)) and sum(sizes_DSBs) > 0 and not any(np.isnan(sizes_SSBs)) and sum(sizes_SSBs) > 0:
                plt.figure(figsize=(14, 7))
                
                # Biểu đồ hình tròn cho DSBs (bên trái)
                plt.subplot(1, 2, 1)
                labels_DSBs = ['Direct', 'Indirect', 'Hybrid']
                plt.pie(sizes_DSBs, labels=labels_DSBs, autopct='%1.1f%%', startangle=140)
                plt.title(f"DSBs for {os.path.basename(file_path)}")
                
                # Biểu đồ hình tròn cho SSBs (bên phải)
                plt.subplot(1, 2, 2)
                labels_SSBs = ['Direct', 'QuasiDirect', 'Indirect']
                plt.pie(sizes_SSBs, labels=labels_SSBs, autopct='%1.1f%%', startangle=140)
                plt.title(f"SSBs for {os.path.basename(file_path)}")
                
                # Hiển thị biểu đồ
                plt.tight_layout()
                plt.savefig(os.path.join(folder_path, f"DSBs_SSBs_PieChart_{os.path.basename(file_path).replace('.phsp', '')}.png"))
                plt.close()

    # Tạo DataFrame từ bảng so sánh
    comparison_df = pd.DataFrame(results)
    
    # Hiển thị hộp thoại để chọn đường dẫn lưu tệp Excel ngay lập tức sau khi xử lý xong
    output_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
    
    if output_path:
        # Tạo file Excel từ DataFrame
        comparison_df.to_excel(output_path, index=False)
        print(f"Excel file saved to: {output_path}")
        label.config(text=f"Result saved to: {output_path}")
    else:
        label.config(text="Save operation was canceled.")

# Tạo cửa sổ chính của Tkinter
root = Tk()
root.title("Analyze Phase Space Files")

# Thiết lập kích thước cố định cho cửa sổ
root.geometry("500x250")  
root.resizable(False, False)  

# Thêm nút và nhãn vào giao diện
label = Label(root, text="Select Phase Space Files (.phsp)", font=("Arial", 12), anchor='center', justify='center')
label.pack(pady=10)

button = Button(root, text="Select and Analyze Files", command=analyze_files, font=("Arial", 12))
button.pack(pady=20)

# Chạy vòng lặp chính của Tkinter
root.mainloop()


Excel file saved to: //wsl$/Ubuntu-20.04/home/tduong/Applications/thanhtai/TbLuCuSc/SBSDamage/TestCellStructure/Lu177/StepByStep/StepByStep_summary2.xlsx
